In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2022 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10692\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,J A N U A R Y,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward Drake (director/screenplay); Timothy V....,[5],NaN
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6],NaN
...,...,...,...,...,...,...,...
325,D E C E M B E R,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,NaN,[265]
326,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN
327,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN
328,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_2021 = df[['Title','Cast and crew']]

In [11]:
df_2021

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,American Siege,Edward Drake (director/screenplay); Timothy V....
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
...,...,...
325,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...
326,NaN,NaN
327,NaN,NaN
328,NaN,NaN


In [12]:
!pip install tmdbv3api

Defaulting to user installation because normal site-packages is not writeable


In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '71b41c0bf036bd0a8917fbc020e0a79d'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [15]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10692\3587185986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))


In [16]:
df_2021

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Thriller Crime Drama
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
...,...,...,...
325,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama
326,NaN,NaN,NaN
327,NaN,NaN,NaN
328,NaN,NaN,NaN


In [17]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [18]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10692\3118254134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))


In [19]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [20]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10692\1611661901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [21]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [22]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10692\3001516348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [23]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [24]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [25]:
df_2021

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Thriller Crime Drama,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
...,...,...,...,...,...,...,...
325,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick
326,NaN,NaN,NaN,nan,nan,NaN,NaN
327,NaN,NaN,NaN,nan,nan,NaN,NaN
328,NaN,NaN,NaN,nan,nan,NaN,NaN


In [26]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [27]:
new_df20 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [28]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN,Action Thriller Crime Drama,American Siege
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
...,...,...,...,...,...,...
325,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Drama,"Alice, Darling"
326,nan,nan,NaN,NaN,NaN,NaN
327,nan,nan,NaN,NaN,NaN,NaN
328,nan,nan,NaN,NaN,NaN,NaN


In [29]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [30]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name     11
actor_3_name     37
genres            5
movie_title       4
comb             38
dtype: int64

In [31]:
new_df20 = new_df20.dropna(how='any')

In [32]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [33]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10692\2267385682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [34]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
5,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Comedy Family Fantasy Adventure,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Jennife...
6,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug,Thriller,shattered,Cameron Monaghan Frank Grillo Lilly Krug Luis ...
...,...,...,...,...,...,...,...
321,Damien Chazelle,Brad Pitt,Margot Robbie,Diego Calva,Drama Comedy,babylon,Brad Pitt Margot Robbie Diego Calva Damien Cha...
322,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime Thriller Mystery Horror,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
323,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Drama Comedy,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...
324,Juel Taylor,John Boyega,Teyonah Parris,Kiefer Sutherland,Science Fiction Comedy Mystery,they cloned tyrone,John Boyega Teyonah Parris Kiefer Sutherland J...


In [35]:
old_df = pd.read_csv('new5_data.csv')

In [36]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6613,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6614,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
6615,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
6616,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [37]:
final_df = old_df.append(new_df20,ignore_index=True)

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10692\3205022339.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df20,ignore_index=True)


In [38]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6905,Damien Chazelle,Brad Pitt,Margot Robbie,Diego Calva,Drama Comedy,babylon,Brad Pitt Margot Robbie Diego Calva Damien Cha...
6906,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime Thriller Mystery Horror,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
6907,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Drama Comedy,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...
6908,Juel Taylor,John Boyega,Teyonah Parris,Kiefer Sutherland,Science Fiction Comedy Mystery,they cloned tyrone,John Boyega Teyonah Parris Kiefer Sutherland J...


In [39]:
final_df.to_csv('new6_data.csv',index=False)